In [44]:
//input
val fileName1 = "dataset_abstracts/a9000038.txt"
val fileName2 = "dataset_abstracts/a9000040.txt"

val rows1 = sc.textFile(fileName1).map(line=>line.trim().replaceAll("(\\s)+", " ")).cache()
val text1 = rows1.reduce(_ + _)

val rows2 = sc.textFile(fileName2).map(line=>line.trim().replaceAll("(\\s)+", " ")).cache()
val text2 = rows2.reduce(_ + _)

fileName1 = dataset_abstracts/a9000038.txt
fileName2 = dataset_abstracts/a9000040.txt
rows1 = MapPartitionsRDD[5756] at map at <console>:41
text1 = principal investigator and associates. Topics in the followingareas are to be considered: (1) controlled Markov diffusionsand nonlinear PDEs; (2) asymptotic properties of nearlydeterministic Markov processes; (3) financial economicsapplications; (4) singular stochastic control; (5) computationalmethods in stochastic control; (6) stochastic calculus ofvariations; (7) nonlinear estimation. Analytical methods basedon viscosity solution techniques for nonlinear differentialequations as well as probabilistic methods will be studied.These theoretical studies are the basis for applied proble...


lastException: Throwable = null


principal investigator and associates. Topics in the followingareas are to be considered: (1) controlled Markov diffusionsand nonlinear PDEs; (2) asymptotic properties of nearlydeterministic Markov processes; (3) financial economicsapplications; (4) singular stochastic control; (5) computationalmethods in stochastic control; (6) stochastic calculus ofvariations; (7) nonlinear estimation. Analytical methods basedon viscosity solution techniques for nonlinear differentialequations as well as probabilistic methods will be studied.These theoretical studies are the basis for applied proble...

In [45]:
//k-gram
val k = 5
val k_gram1 = text1.split("").sliding(k).toList.map(x => (scala.util.hashing.MurmurHash3.arrayHash(x), ""))
val kgramRDD1 = sc.parallelize(k_gram1)
val filtered1 = kgramRDD1.distinct()

val k_gram2 = text2.split("").sliding(k).toList.map(x => (scala.util.hashing.MurmurHash3.arrayHash(x), ""))
val kgramRDD2 = sc.parallelize(k_gram2)
val filtered2 = kgramRDD2.distinct()


k = 5
k_gram1 = List((323183978,""), (879426833,""), (207652440,""), (1023262041,""), (1191319030,""), (-1029485329,""), (94755376,""), (798376504,""), (-1163905680,""), (982395043,""), (-487295416,""), (929914478,""), (1998737565,""), (2078865377,""), (-1593058553,""), (-81630221,""), (1675405219,""), (-1975782702,""), (-312516106,""), (-2146848343,""), (899011444,""), (-2008099792,""), (1597990745,""), (1159842749,""), (2046055236,""), (-1557845639,""), (-2100210802,""), (-206169076,""), (-19109906,""), (1127829784,""), (-121854170,""), (772595528,""), (734667765,""), (-1813676078,""), (-708552550,""), (-1949504179,""), (-707004830,""), (179624194,""), (-1471358333,""), (45318084,""), (2014423945,""), (-618699956,""), (1272547154,""), (240336297,""), (9266125...


List((323183978,""), (879426833,""), (207652440,""), (1023262041,""), (1191319030,""), (-1029485329,""), (94755376,""), (798376504,""), (-1163905680,""), (982395043,""), (-487295416,""), (929914478,""), (1998737565,""), (2078865377,""), (-1593058553,""), (-81630221,""), (1675405219,""), (-1975782702,""), (-312516106,""), (-2146848343,""), (899011444,""), (-2008099792,""), (1597990745,""), (1159842749,""), (2046055236,""), (-1557845639,""), (-2100210802,""), (-206169076,""), (-19109906,""), (1127829784,""), (-121854170,""), (772595528,""), (734667765,""), (-1813676078,""), (-708552550,""), (-1949504179,""), (-707004830,""), (179624194,""), (-1471358333,""), (45318084,""), (2014423945,""), (-618699956,""), (1272547154,""), (240336297,""), (9266125...

In [46]:

val join = filtered1.leftOuterJoin(filtered2)
val join_n = join.count().toFloat
val union = filtered1.union(filtered2).distinct()
val union_n = union.count().toFloat

val sim = join_n / union_n
val jacc_dist = 1-sim

println(sim)
println(jacc_dist)


0.6188993
0.3811007


join = MapPartitionsRDD[5770] at leftOuterJoin at <console>:40
join_n = 1192.0
union = MapPartitionsRDD[5774] at distinct at <console>:42
union_n = 1926.0
sim = 0.6188993
jacc_dist = 0.3811007


0.3811007

Now we work with Min-hashing

In [47]:
import java.io.File

val dir = new File("dataset_abstracts")
val files = dir.listFiles() //379
val maxSize = 10
val docSize = if(files.size < maxSize) files.size else maxSize

val docs = files.map(f => sc.textFile(f.getPath() ).map(line=>line.replaceAll("(\\s)+", " ")).collect().reduce(_ + _))
val docsRDD = sc.parallelize(docs).cache()
val abstracts = docsRDD.map(doc => doc.split("Abstract :")).filter(a => a.size > 1).map(a => a(1))

//TODO RDD[ RDD [String]] --> RDD[(docId, shingle)]
val k = 5
val k_gram1 = abstracts.map(a => a.split("").sliding(k).toList.map(x => (scala.util.hashing.MurmurHash3.arrayHash(x), "")))
val kgramRDD = sc.parallelize(k_gram1)
val filtered = kgramRDD.distinct()

Name: Unknown Error
Message: <console>:55: error: type mismatch;
 found   : org.apache.spark.rdd.RDD[List[(Int, String)]]
 required: Seq[?]
Error occurred in an application involving default arguments.
       val kgramRDD = sc.parallelize(k_gram1)
                                     ^

StackTrace: 

In [48]:
//Min-hashing

val p = 1073676287
val m = p + 1
def hashThis(a:Long, b:Long, x:Long): Long = {
    ((a*x + b) % p ) % m
}

//TODO: Outer loop: foreach document in set of documents
var doc_signature:Array[Long] = new Array[Long](100)
for(i<-0 to 99){
    
    val r = new scala.util.Random(i)
    val a2 = r.nextInt(p-1)
    val a = if(a2%2==0) a2+1 else a2
    val b = r.nextInt(p-1)
    
    //TODO: Iterate through docs
    //for(d in doc){
    //    val min = d.minhashThis()
    //}
    
    val min = filtered1.map(s => s._1.toLong).reduce((x,y) => hashThis(a, b, x) min hashThis(a, b, y) )
    
    doc_signature(i) = min
    
}

p = 1073676287
m = 1073676288
doc_signature = Array(-701099952, -490456252, -819415235, -982796781, -972365587, -539719326, -991711111, -861865276, -954276687, -936137481, -636549802, -291813556, -958014372, -751095575, -226580326, -298989585, -934393359, -761294500, -471033875, -564140757, -786245149, -853592562, -256835389, -955319261, -234031736, -885353197, -422056576, -685021830, -1053303560, -801120782, -538153782, -901692924, -1032942575, -425691361, -263268863, -928073791, -844671812, -845859844, -691309734, -846536457, -1019873399, -456506007, -890476283, -635220461, -700645489, -827812163, -348668524, -727526301, -328853667, -977040893, -933394679, -891249220, -936442531, -636264737, -90962801, -1022817264, -1035...


hashThis: (a: Long, b: Long, x: Long)Long


Array(-701099952, -490456252, -819415235, -982796781, -972365587, -539719326, -991711111, -861865276, -954276687, -936137481, -636549802, -291813556, -958014372, -751095575, -226580326, -298989585, -934393359, -761294500, -471033875, -564140757, -786245149, -853592562, -256835389, -955319261, -234031736, -885353197, -422056576, -685021830, -1053303560, -801120782, -538153782, -901692924, -1032942575, -425691361, -263268863, -928073791, -844671812, -845859844, -691309734, -846536457, -1019873399, -456506007, -890476283, -635220461, -700645489, -827812163, -348668524, -727526301, -328853667, -977040893, -933394679, -891249220, -936442531, -636264737, -90962801, -1022817264, -1035...